In [1]:
%%capture
!pip install plotly
!pip install chart-studio
!pip install -q kaggle

In [40]:
# Work With Files
from google.cloud import storage
import os

# Useful libraries:
from time import time
import numpy as np
import pandas as pd
import math

# To Plot:
import matplotlib.pyplot as plt
import seaborn as sns
import chart_studio.plotly as py
import plotly.graph_objs as go

# Pyspark Lib:
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import col
import pyspark.sql.functions as F
from pyspark import SparkContext, SparkConf


# Preprocess:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import MinMaxScaler

# Pysparl ML:
from pyspark.ml.regression import LinearRegression

In [3]:
PATH_BUCKET = 'gs://nyc_comp_bk/'
PATH_DATA = '/home/ubuntu/NYC_Taxi/data/'

In [4]:
os.chdir('/home/ubuntu/NYC_Taxi/')

In [5]:
class Work_On_Bucket():
    
    def __init__(self, bucket_name):
        # Get access to the bucket:
        storage_client = storage.Client()
        self.bucket = storage_client.get_bucket(bucket_name)
        
    def get_file_from_bucket(self, file_name, save_path):
        # Download the file:
        blob = self.bucket.blob(file_name)
        blob.download_to_filename(''.join([save_path, file_name]))
            
    def upload_file_to_bucket(self, file_name, folder_path):
        # Upload the File
        object_to_save = self.bucket.blob(file_name)
        object_to_save.upload_from_filename(folder_path + file_name)


### Get the Dataset:

In [6]:
Bucket = Work_On_Bucket('nyc_comp_bk')

In [7]:
# Set kaggle:
! mkdir ~/.kaggle
Bucket.get_file_from_bucket('kaggle.json', '/home/ubuntu/NYC_Taxi/')
! cp /home/ubuntu/NYC_Taxi/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
# Download The Dataset
!kaggle competitions download -c new-york-city-taxi-fare-prediction

# Unzip the Files
! unzip new-york-city-taxi-fare-prediction.zip -d /home/ubuntu/NYC_Taxi/data/
! rm new-york-city-taxi-fare-prediction.zip

# Upload databses to bucket:
print('Start Uploding!')
Bucket.upload_file_to_bucket('train.csv', PATH_DATA)
Bucket.upload_file_to_bucket('test.csv', PATH_DATA)
print('Succesfully Uploaded!')

mkdir: cannot create directory ‘/root/.kaggle’: File exists
 99%|█████████████████████████████████████▋| 1.55G/1.56G [00:25<00:00, 56.1MB/s]
100%|██████████████████████████████████████| 1.56G/1.56G [00:25<00:00, 66.2MB/s]
Archive:  new-york-city-taxi-fare-prediction.zip
  inflating: /home/ubuntu/NYC_Taxi/data/GCP-Coupons-Instructions.rtf  
  inflating: /home/ubuntu/NYC_Taxi/data/sample_submission.csv  
  inflating: /home/ubuntu/NYC_Taxi/data/test.csv  
  inflating: /home/ubuntu/NYC_Taxi/data/train.csv  


### Preliminary Steps (Load + Checks):

In [8]:
# Load Data:
train = spark.read.load(PATH_BUCKET+"train.csv", format="csv", inferSchema="true", header="true")
test = spark.read.load(PATH_BUCKET+"test.csv", format="csv", inferSchema="true", header="true")

# Load Test (Because pyspark changes the timestamp):
Bucket.get_file_from_bucket('test.csv', '')
original_test = pd.read_csv('test.csv')

In [10]:
# Get DB shape:
ncol = len(train.columns)
nrow = train.count()
print("The shape of the dataset is {:d} rows by {:d} columns".format(nrow, ncol))

The shape of the dataset is 55423856 rows by 8 columns


In [8]:
# Get the schema:
train.printSchema()

root
 |-- key: timestamp (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- passenger_count: integer (nullable = true)



In [25]:
# Show some basic Statistics:
stats = train.select(train.columns[1:]).describe()
stats.toPandas()

,summary,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,count,55423856,55423856,55423856,55423856,55423480,55423480,55423856
1,mean,11.345045601663852,None,-72.50968444358728,39.919791786888176,-72.5112097297181,39.92068144482884,1.6853799201556816
2,stddev,20.7108321982325,None,12.848883381402652,9.642353041994932,12.782196517830771,9.633345796415126,1.327664357095968
3,min,-300.0,2009-01-01 00:00:27 UTC,-3442.059565,-3492.263768,-3442.024565,-3547.886698,0
4,max,93963.36,2015-06-30 23:59:54 UTC,3457.625683,3408.789565,3457.62235,3537.132528,208


In [12]:
# Check Nulls:
for c in train.columns[2:]:
    nans = train.where(col(c).isNull()).count()
    print('{:s}: {:d}'.format(c, nans))

pickup_datetime: 0
pickup_longitude: 0
pickup_latitude: 0
dropoff_longitude: 376
dropoff_latitude: 376
passenger_count: 0


In [9]:
# Remove Rows with Missing Values:
train = train.na.drop(how='any')

In [9]:
# Check Duplicates:
print('The Duplicates are: {:d}'.format(train.count()-train.distinct().count()))

The Duplicates are: 1650


In [10]:
# Remove Duplicates:
train = train.distinct()

### Work With the Fare Price (Dependent Variale):

Checking the statistics, it is clear that some values are not possible, I am going to remove negatives values and all values that are lower that 2.50 dollars (https://nymag.com/nymetro/urban/features/taxi/n_20286/). Hence, I will keep all values greather than 2.50 dollars. By taking into account the max value, it is clear that some outlayers are present, nobady would want to spend 93,963 dollars for a trip. Working with the quantiles it is clear that there is somenthing wrong with the fare amount, the median is 999.9 dollars. As starting point let's remove what I am pretty sure that is an outlayer, namely everything over the fourth quntile, which represents all values greater than 93963.36 dollars. Computing again the quantiles now the fourth quantile is 999.99 dollars, Checking the other variables associated with these high fare amounts, it comes up that some of them doesn't make any sense: no changes in lat and long pick up and drop off, too small journey with one or two people for a so high price, missing values for lat and long and so on. I am going to keep al observations with fear amount less than 999.99 dollars, hopping that with the EDA I can clean more the data.

In [12]:
# Get statistics:
train.select('fare_amount').describe().toPandas()

,summary,fare_amount
0,count,55421830
1,mean,11.345105672079605
2,stddev,20.711131158405834
3,min,-300.0
4,max,93963.36


In [11]:
# Remove Values lower than 2.50$:
train = train.filter('fare_amount > 2.50')

In [52]:
# Compute Quantiles:
train.approxQuantile('fare_amount', [0.25, 0.50, 0.75, 0.975], 0.25)

[2.51, 400.0, 93963.36, 93963.36]

In [12]:
# Remove Outlayers:
train = train.filter('fare_amount < 93963.36')

In [54]:
# Compute Again Quantiles
train.approxQuantile('fare_amount', [0.25, 0.50, 0.75, 0.975], 0.25)

[2.51, 419.75, 75747.02, 75747.02]

In [13]:
# Remove other Outlayers:
train = train.filter('fare_amount < 999.99')

In [56]:
# Check Quantiles:
train.approxQuantile('fare_amount', [0.25, 0.50, 0.75, 0.975], 0.25)

[2.51, 400.0, 978.0, 978.0]

### Create My Base Line:

As Base Line I am going to use a Multiple Linear Regression that takes as input all the scaled (mean=0, sd=1) numerical variables.
As result I get an RMSE = 9.40719 on the Test.

In [14]:
NUMERICAL = ['pickup_longitude',
            'pickup_latitude',
            'dropoff_longitude',
            'dropoff_latitude',
            'passenger_count']
TARGET = 'fare_amount'

In [59]:
# Get the feature Vector:
assembler = VectorAssembler(inputCols=NUMERICAL, outputCol="features")
train_df = assembler.transform(train)
train_df = train_df.select('features', TARGET)

In [60]:
# Scale Data:
scaler = StandardScaler(inputCol="features", outputCol="ScaledFeatures")
scalerModel = scaler.fit(train_df)
train_df = scalerModel.transform(train_df)
train_df = train_df.select('ScaledFeatures', TARGET)

In [61]:
# Run the Linear Regression:
lr = LinearRegression(featuresCol="ScaledFeatures", labelCol=TARGET, maxIter=10)
lr_model = lr.fit(train_df)

# Print Stats:
training_result = lr_model.summary
print("***** Training Set *****")
print("RMSE: {:.3f}".format(training_result.rootMeanSquaredError))
print("MAE: {:.3f}".format(training_result.meanAbsoluteError))
print("R2: {:.3f}".format(training_result.r2))
print("***** Training Set *****")

***** Training Set *****
RMSE: 9.792
MAE: 6.030
R2: 0.000
***** Training Set *****


In [63]:
# Prepare the test:
test_df = assembler.transform(test.select(NUMERICAL))
test_df = test_df.select('features')
test_df = scalerModel.transform(test_df)

# Make Predictions:
predictions = lr_model.transform(test_df).select('prediction').withColumnRenamed('prediction','fare_amount').toPandas()

# Prepare the Submission:
submission = pd.concat([original_test['key'], predictions['fare_amount']], axis=1)
submission.to_csv('submission.csv', index=False)

# Submit:
!kaggle competitions submit -c new-york-city-taxi-fare-prediction -f submission.csv -m "First Submission"
!kaggle competitions submissions -c new-york-city-taxi-fare-prediction

100%|█████████████████████████████████████████| 459k/459k [00:02<00:00, 228kB/s]
Successfully submitted to New York City Taxi Fare PredictionfileName        date                 description       status    publicScore  privateScore  
--------------  -------------------  ----------------  --------  -----------  ------------  
submission.csv  2021-06-26 11:42:04  First Submission  complete  9.40719      9.40719       
submission.csv  2021-06-23 15:15:57  First Submission  complete  9.40712      9.40712       
submission.csv  2021-06-23 15:10:56  First Submission  complete  9.40712      9.40712       
submission.csv  2021-06-23 14:58:58  None              error     None         None          
submission.csv  2021-06-23 14:58:06  First Submission  error     None         None          
submission.csv  2021-06-23 14:55:46  None              error     None         None          
submission.csv  2021-06-23 14:54:33  First Submission  error     None         None          
submission.csv  2021-0

### EDA

#### 1) Latitude and Longitude

From the study of the quantiles it is clear that is plenty of outlayers, because the NYC coordinates are (lat=40.730610 lon=-73.935242). As first step, it is possible to remove everything that is out of the I and III which are very extrem values (as we can see). 

The it is possible to compute new features that can be useful both for the model and to remove more outlayers, namely the Absolute Distance, the Haversinee Distance and the Minkowski Distance (that for p=1 is the Manhattan and for p=2 is the Euclidean distance). All the distances are expressed in kilometers. To obtain some of them I need to know to how much km corresponds the variation of 1 degree for each coordinates and it turns put that for Latitude it is 111 Km and for Longitude is 85 Km.

In [20]:
# Check I, II and III Quantiles:
np.array([train.approxQuantile('pickup_longitude', [0.25, 0.50, 0.75], 0.25),
          train.approxQuantile('pickup_latitude', [0.25, 0.50, 0.75], 0.25),
          train.approxQuantile('dropoff_longitude', [0.25, 0.50, 0.75], 0.25),
          train.approxQuantile('dropoff_latitude', [0.25, 0.50, 0.75], 0.25)])

array([[-3442.059565,  2084.697707,  3457.625683],
       [-3492.263768,  1963.515858,  3408.789565],
       [-3442.024565,   -73.979903,  3457.62235 ],
       [-3547.886698,  3015.889707,  3537.132528]])

In [15]:
# Remove any observation out of the I and III quantile, for each lat/long variable:
train = train.filter((col('pickup_longitude') > -3442.059565) |
             (col('pickup_longitude') < 3457.625683) |
             (col('pickup_latitude') > -3492.263768) |
             (col('pickup_latitude') < 3408.789565) |
             (col('dropoff_longitude') > -3442.024565) |
             (col('dropoff_longitude') < 3457.62235) |
             (col('dropoff_latitude')  >-3547.886698) |
             (col('dropoff_latitude') < 3537.1325288)
            )

In [16]:
# Write the function to compute the distances:

class Coordinates_Transform():

    def __init__(self, df):

        self.df = df
        self.cols = df.columns
        self.haversine_udf = F.udf(Coordinates_Transform.haversine_stat, DoubleType())
        self.airports_udf = F.udf(Coordinates_Transform.identify_airports, StringType())

    @staticmethod
    def haversine_stat(pick_lat, drop_lat, pick_long, drop_long):

        longit_a, latit_a, longit_b, latit_b = map(math.radians, [pick_long,  pick_lat, drop_long, drop_lat])
        dist_longit = longit_b - longit_a
        dist_latit = latit_b - latit_a
        # Calculate area
        area = math.sin(dist_latit/2)**2 + math.cos(latit_a) * math.cos(latit_b) * math.sin(dist_longit/2)**2
        # Calculate the central angle
        central_angle = 2 * math.asin(math.sqrt(area))
        RADIUS = 6371
        # Calculate Distance
        distance = central_angle * RADIUS
        return round(distance, 3)

    def add_dist_metrics(self):

        # Absolute Lat e Long Distance:
        self.df = self.df.withColumn( 'abs_dist_longitude', F.round( F.abs( col('pickup_longitude') - col('dropoff_longitude') ) * 85, 3 ) )
        self.df = self.df.withColumn( 'abs_dist_latitude', F.round( F.abs( col('pickup_latitude') - col('dropoff_latitude') ) * 111 , 3 ) )

        # Manhattan Distance:
        self.df = self.df.withColumn( 'manhattan_dist', F.round( col('abs_dist_longitude') + col('abs_dist_latitude'), 3 ) )

        # Euclidean Distance:
        self.df = self.df.withColumn( 'euclidean_dist',  F.round( F.sqrt( col('abs_dist_longitude')**2 + col('abs_dist_latitude')**2 ), 3 ) )

        # Haversine Distance:
        self.df = self.df.withColumn( 'haversine_dist', 
                                     self.haversine_udf( col('pickup_latitude'), col('dropoff_latitude'), col('pickup_longitude'), col('dropoff_longitude') ) ) 

        return self.df

    @staticmethod
    def identify_airports(pick_lat, drop_lat, pick_long, drop_long):

        # Set the coordinates of airports:
        JFK_LAT = 40.641766
        JFK_LON = -73.780968

        LGR_LAT = 40.773013
        LGR_LON = -73.870229

        EWR_LAT = 40.689531
        EWR_LON = -74.174462

        # Compute distances:
        pick_up_jfk = Coordinates_Transform.haversine_stat(JFK_LAT, pick_lat, JFK_LON, pick_long)
        drop_off_jfk = Coordinates_Transform.haversine_stat(drop_lat, JFK_LAT, drop_long, JFK_LON)

        pick_up_lgr = Coordinates_Transform.haversine_stat(LGR_LAT, pick_lat, LGR_LON, pick_long)
        drop_off_lgr = Coordinates_Transform.haversine_stat(drop_lat, LGR_LAT, drop_long, LGR_LON)

        pick_up_ewr = Coordinates_Transform.haversine_stat(EWR_LAT, pick_lat, EWR_LON, pick_long)
        drop_off_ewr = Coordinates_Transform.haversine_stat(drop_lat, EWR_LAT, drop_long, EWR_LON)

        # Assign a value:
        if pick_up_jfk < 1:
            return 'PICK_JFK'
        elif drop_off_jfk < 1:
            return 'DROP_JFK'
        if pick_up_lgr < 1:
            return 'PICK_LGR'
        elif drop_off_lgr < 1:
            return 'DROP_LGR'
        if pick_up_ewr < 1:
            return 'PICK_EWR'
        elif drop_off_ewr < 1:
            return 'DROP_EWR'
        else:
            return 'NO_AIRPORT'
  
      def airports(self):

        self.df = self.df.withColumn('airport',
                                     self.airports_udf( col('pickup_latitude'), col('dropoff_latitude'), col('pickup_longitude'), col('dropoff_longitude') ))
        return self.df
    
      def long_short_trip(self):
    
        quantiles = self.df.approxQuantile('haversine_dist', [0.5, 0.75], 0.25)
        med = quantiles[0]
        third = quantiles[1]
        self.df = self.df.withColumn( 'long_short', F.when(col('haversine_dist') <= med, 'SHORT') \
                                     .when( ( col('haversine_dist') > med ) & ( col('haversine_dist') < third ), 'LONG' ) \
                                    .otherwise('VERY_LONG') )
        return self.df
    

In [ ]:
# Get the new features.

coordTransform = Coordinates_Transform(train)
train = coordTransform.add_dist_metrics()
train = coordTransform.airports()
train = coordTransform.long_short_trip()